# Connecting our Models

### Introduction

In this lesson, we'll see how we can use our objects to return a list of object instances instead of database records.  Let's get started.

### Using an Object

Now so far, we have been successful at connecting a request to our API to a request to our database.  And we did so with code like the following:

```python
# application.py
import sqlite3
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/nhl/players/')
def show_players():
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players;')
    players = cursor.fetchall()
    return jsonify(players)

app.run(debug = True)
```

Now when we make a request to the `/nhl/players` url, each player is represented as a list.

<img src="./nhl_players_json.png" width="40%">

It would be better if our players were represented as dictionaries.  

Perhaps one way that we can get those list of dictionaries is to create a list of *player objects* with our return values from the database.  

So this means that we would like to retrieve a record from the database, and for each record pass the through the data to create a corresponding instance.

Let's see how we can do this.

### Connecting to our database

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/apis-jigsaw/flask-models-reading/main/nhl_players.csv', index_col = 0)

df[:2]

,name,age,ht,wt,shot,birth_place,birthdate,number
id,,,,,,,,
0,Connor Bunnaman,21,"6' 1""",207 lbs,L,"Guelph, ON",04/16/98,82
1,Sean Couturier,27,"6' 3""",211 lbs,L,"Phoenix, AZ",12/07/92,14


In [10]:
df.to_sql('players', conn, if_exists = 'replace')

In [11]:
import sqlite3
conn = sqlite3.connect('nhl.db')
cursor = conn.cursor()
cursor.execute('select * from players limit 1;')
player_record = cursor.fetchone()

In [12]:
player_record

(0,
 'Connor Bunnaman',
 21,
 '6\' 1"',
 '207 lbs',
 'L',
 'Guelph, ON',
 '04/16/98',
 82)

Then, once we have this data, we can create a Player object with this data,  with something like the following.

In [13]:
class Player:
    def __init__(self, values):
        self.values = values

In [15]:
player = Player(player_record)
player.values

(0,
 'Connor Bunnaman',
 21,
 '6\' 1"',
 '207 lbs',
 'L',
 'Guelph, ON',
 '04/16/98',
 82)

Now of course, what we would really like is to set each value to a corresponding attribute.  We can accomplish that, with something like the following:

In [20]:
class Player:

    columns = ['id', 'name', 'age', 'height', 'weight', 
               'shot', 'birth_place', 'birthdate', 'number']
    
    def __init__(self, values):
        self.__dict__ = dict(zip(self.columns, values))

Now let's see what happens when we pass in our values.

In [18]:
player = Player(player_record)
player.__dict__

{'id': 0,
 'name': 'Connor Bunnaman',
 'age': 21,
 'height': '6\' 1"',
 'weight': '207 lbs',
 'shot': 'L',
 'birth_place': 'Guelph, ON',
 'birthdate': '04/16/98',
 'number': 82}

So we can see that this created a player with the proper attributes.  Let's take another look at how this worked.

In [19]:
class Player:

    columns = ['id', 'name', 'age', 'height', 'weight', 
               'shot', 'birth_place', 'birthdate', 'number']
    
    def __init__(self, values):
        self.__dict__ = dict(zip(self.columns, values))

If we focus on the `__init__` function, self.columns is the list of columns, which we then align with the list of data that we retrieve from the database.  We then convert that data into a dictionary, and assign it as attributes of the object with the `__dict__` property.

### A last step

As a last step, remember that we have written a function in our orm to perform the proper `SELECT *` statement.  This is our `find_all` function.  Let's try using that here.

```python
@app.route('/nhl/players/')
def show_players():
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    player_records = cursor.execute('select * from players;')
    players = [Player(player) for player_records in player_records]
    player_dicts = [player.__dict__ for player in players]
    return jsonify(player_dicts)
```

So we can see that we are able to retrieve a list of player instances from the database.  Then we turned these records into Player instances.  And finally, we needed to convert those instances into dictionaries with the `player.__dict__` attribute, so we could ultimately send the data back as json.

### Summary

In this lesson, we saw how we can take data from a database and use it to create corresponding object instances.  We then needed to convert the data back into dictionaries to ultimately send back the data as json.

We spent sometime understanding how we could convert our players into instances.  Let's take another look at the code.

```python
class Player:

    columns = ['id', 'name', 'age', 'height', 'weight', 
               'shot', 'birth_place', 'birthdate', 'number']
    
    def __init__(self, values):
        self.__dict__ = dict(zip(self.columns, values))
```

Above we pair together the tuple of values with a the corresponding columns.  Then we turn that paired data into a dictionary and assign the instance's attributes with the `__dict__` property.